# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [2]:
# TODO: Install any packages that you might need
!pip install smdebug

  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)


In [3]:
# TODO: Import any packages that you might need
from tqdm import tqdm
import os
from sagemaker.debugger import Rule, DebuggerHookConfig, ProfilerConfig, ProfilerRule, rule_configs, FrameworkProfile
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch


## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [8]:
import os
import json
import boto3

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

  0%|          | 0/1228 [00:00<?, ?it/s]

  0%|          | 1/2299 [00:00<05:52,  6.52it/s]

  0%|          | 1/2666 [00:00<05:38,  7.87it/s]

  0%|          | 2/2373 [00:00<03:41, 10.71it/s]

  0%|          | 2/1875 [00:00<02:13, 14.08it/s]

100%|██████████| 1875/1875 [03:05<00:00, 10.12it/s]


## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [2]:
#TODO: Perform any data cleaning or data preprocessing
from os import path, listdir, makedirs
import glob
from tqdm import tqdm

def directory_to_dictionay(directory):
    final_dict = {}
    
    
    
def train_test_split(dataset_path, per_test, per_val):
    classes_directory = listdir(dataset_path)
    result_classes_split = {'train':{ class_dir: [] for class_dir in classes_directory},
                                            'val':{ class_dir: [] for class_dir in classes_directory},
                                           'test':{ class_dir: [] for class_dir in classes_directory}}
    if per_test + per_val >= 100:
        raise Exception( 'ERROR')
    
    for class_directory in classes_directory:
        elements_class =  glob.glob(f'{path.join(dataset_path, class_directory)}/*')
        n_elements_class = len(elements_class)
        n_sample_test  =  (n_elements_class * per_test) //100
        n_sample_val  =  (n_elements_class * per_val) //100
        result_classes_split['test'][class_directory] = elements_class[:n_sample_test -1 ]
        result_classes_split['val'][class_directory] = elements_class[n_sample_test - 1 : n_sample_test  + n_sample_val - 2]
        result_classes_split['train'][class_directory] = elements_class[n_sample_test  + n_sample_val - 2 : ]
        # length = len(result_classes_split['test'][class_directory] ) + len(result_classes_split['val'][class_directory])  + len(result_classes_split['train'][class_directory])
        # assert length == len(elements_class)  ,'convertion error'
        
    return result_classes_split

def write_sample_elements(dir_name, data):
    makedirs(dir_name, exist_ok= True)
    classes = list(data.keys())
    for class_name in classes:
        dir_class = path.join(dir_name, class_name)
        makedirs(dir_class, exist_ok=True)
        for file_path in tqdm(data[class_name]):
            filename_output = path.join(path.join(dir_name, class_name), file_path.split('/')[-1])
            with open(file_path, 'rb') as img_input, open(filename_output, 'wb') as img_output:
                img_output.write(img_input.read())

In [3]:
splitted_data = train_test_split('train_data/', 20, 10)
base_directory = 'BinImages'
for dataset_name  in splitted_data:
    dir_name = path.join(base_directory,  dataset_name)    
    write_sample_elements(dir_name,  splitted_data[dataset_name])

100%|██████████| 532/532 [00:08<00:00, 59.47it/s]


In [5]:
splitted_data.keys()

dict_keys(['train', 'val', 'test'])

In [7]:
%%capture
!aws s3 cp  BinImages  s3://lucialedezmacapstoneproject/BinImages/  --recursive

# import sagemaker
# bucket = "lucialedezmacapstone"
# sagemaker_session = sagemaker.Session()
# inputs = sagemaker_session.upload_data(path="train_data", bucket=bucket, key_prefix="train_data_2")

## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [10]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values
hyperparameters = {'batch-size':  64,  'lr':  0.009, 'model-arch':  'resnet50', 'model-n-classes': 5,  'epochs': 6}

In [11]:
#TODO: Create your training estimator

role = get_execution_role()

estimator = PyTorch(
    entry_point='train.py',
    base_job_name='bin-images-estimator',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.4.0',
    py_version='py3',
    hyperparameters=hyperparameters)

In [9]:
# TODO: Fit your estimator
os.environ['SM_CHANNEL_TRAINING'] = 's3://lucialedezmacapstoneproject/BinImages/train'
os.environ["SM_CHANNEL_VALIDATION"] = 's3://lucialedezmacapstoneproject/BinImages/val'
os.environ["SM_CHANNEL_TEST"] = 's3://lucialedezmacapstoneproject/BinImages/test'
os.environ['SM_MODEL_DIR']='s3://lucialedezmacapstoneproject/model/'

estimator.fit({'training': 's3://lucialedezmacapstoneproject/BinImages/train', 
                      'test': 's3://lucialedezmacapstoneproject/BinImages/test',
                       'validation': 's3://lucialedezmacapstoneproject/BinImages/val'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: bin-images-estimator-2023-04-14-21-45-21-520


2023-04-14 21:45:29 Starting - Starting the training job...
2023-04-14 21:45:43 Starting - Preparing the instances for training...
2023-04-14 21:46:25 Downloading - Downloading input data......
2023-04-14 21:47:26 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-14 21:47:37,362 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-14 21:47:37,366 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-04-14 21:47:37,377 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-14 21:47:37,379 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-04-14 21:47:37,542 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2023-04-14 21:47:37,542 sagemaker-conta

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128]),
}
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": objective_metric_name , "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [ ]:
#TODO: Create your training estimator
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
role = get_execution_role()
estimator = PyTorch(
                    entry_point="train.py",
                    role=role,
                    py_version="py3",
                    framework_version="1.8",
                    instance_count=1,
                    instance_type="ml.m5.large"
                    )# TODO: Your estimator here

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [ ]:
# TODO: Fit your estimator
tuner.fit('train': 's3://lucialedezmacapstoneproject/BinImages/ train', 
               'test': 's3://lucialedezmacapstoneproject/BinImages/test',
               'val': 's3://lucialedezmacapstoneproject/BinImages/val',)

In [ ]:
# TODO: Find the best hyperparameters
best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model
best_hyperparameters = best_estimator.hyperparameters()
best_hyperparameters 

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

role = get_execution_role()

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    #Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]


hook_config = DebuggerHookConfig(
    hook_parameters={"include_regex": "CrossEntropyLoss_output.*",
                     "train.save_interval": "4",
                     "eval.save_interval": "2",
                     "predict.save_interval": "4"}
)
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

In [ ]:
# TODO: Create and fit an estimator
estimator = PyTorch(
                    entry_point="train_hook.py",
                    base_job_name="smdebugger-capstone-bin-images",
                    role=role,
                    py_version="py3",
                    framework_version="1.8",
                    instance_count=1,
                    instance_type="ml.m5.large",
                    hyperparameters= best_hyperparameters ,
                    rules=rules,
                    debugger_hook_config=hook_config,
                    profiler_config=profiler_config
                    ) # TODO: Your estimator here
estimator.fit('train': 's3://lucialedezmacapstoneproject/BinImages/ train', 
                       'test': 's3://lucialedezmacapstoneproject/BinImages/test',
                       'val': 's3://lucialedezmacapstoneproject/BinImages/val')

In [ ]:
# TODO: Plot a debugging output.
rom smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals

trial = create_trial(estimator.latest_job_debugger_artifacts_path()) ## we can directly wrute the s3 debugg path
# s3://sagemaker-us-east-1-996862753977/smdebugger-project3-img-classification-2023-03-02-06-23-53-539/debug-output

trial.tensor_names()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot


def plot_tensor(trial, tensor_name):

    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")
    steps_pred, vals_pred = get_data(trial, tensor_name, mode=ModeKeys.PREDICT)
    print("loaded TEST data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()
    par_t = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    par_t.set_xlabel("Steps (PREDICT)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    (p3,) = par.plot(steps_pred, vals_pred, label="pred_" + tensor_name)
    print("completed PREDICT plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())
    
    par_t.xaxis.get_label().set_color(p3.get_color())
    leg.texts[2].set_color(p3.get_color())

    plt.ylabel(tensor_name)

    plt.show()


In [ ]:
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)), 
      len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)),
      len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.PREDICT)))

In [ ]:
plot_tensor(trial, "CrossEntropyLoss_output_0")

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint
predictor=estimator.deploy(initial_instance_count=1, 
    instance_type='ml.m5.large') # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint
from train_model import create_data_loaders

test_loader = create_data_loaders('dogImages/test', 4)
image_loader = None
for images,labels in test_loader:
    break 

# #use the predictor to make a prediction
inference = predictor.predict(images)


In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances

estimator = PyTorch(
                    entry_point="train.py",
                    base_job_name="bin-images-multiinstance-capstone",
                    role=role,
                    py_version="py3",
                    framework_version="1.8",
                    instance_count=5,
                    instance_type="ml.m5.large",
                    hyperparameters=best_hyperparameters ,
                    rules=rules,
                    debugger_hook_config=hook_config,
                    profiler_config=profiler_config
                    ) # TODO: Your estimator here
estimator.fit('train': 's3://lucialedezmacapstoneproject/BinImages/ train', 
                       'test': 's3://lucialedezmacapstoneproject/BinImages/test',
                       'val': 's3://lucialedezmacapstoneproject/BinImages/val')
